# SETUP

The first part of the code sets up the pca_wavelet network, the training comes later. Most of this code comes from the original authors

In [2]:
import sys
import time
sys.path.append('../segmentation_helper')

import tensorflow as tf
import keras
import tqdm
import matplotlib.pyplot as plt
import numpy as np
import data_loader as dl
import model_broker as mb
import os
import pandas as pd
import seaborn as sns

tf.keras.backend.set_floatx("float64")

GPU device not found
Found GPU at: 


In [3]:
def scaledtanh(x): 
    return tf.math.tanh(x*0.5)

def scaledatanh(x):
    return tf.math.atanh(x)*2.0


In [4]:
def conv_calculate_A_and_b(imghead, seghead, img_train,seg_train):
    n = 0.0
    
    imdecom_shape = imghead(next(iter(img_train))[0]).shape
    img_channels = imdecom_shape[3] # shape_0
    imdecom_2d_shape = imdecom_shape[1]*imdecom_shape[2] #shape_1
    
    segdecom_shape = seghead(next(iter(seg_train))[0]).shape
    seg_channels = segdecom_shape[3] # shape_0
    segdecom_2d_shape = segdecom_shape[1]*segdecom_shape[2] #shape_1
    

    xxt = np.zeros([img_channels,img_channels])
    yxt = np.zeros([img_channels,seg_channels])
    x = np.ones([imdecom_2d_shape])
    x_m = np.zeros([img_channels])
    y = np.ones([segdecom_2d_shape]) 
    y_m = np.zeros([seg_channels])

    bar = tqdm.notebook.tqdm(total = int(img_train.cardinality()))

    for item in iter(zip(img_train,seg_train)):
        bar.update(1)
        image = item[0][0]
        segmentation = item[1][0]

        imgdecom = imghead(image)
        segdecom = seghead(segmentation)

        mat = tf.reshape(imgdecom,[-1,seg_channels])
        segmat = tf.reshape(segdecom,[-1,img_channels])

        cov = tf.tensordot(mat,mat,[0,0])
        xxt += cov
        #del cov

        segcov = tf.tensordot(mat,segmat,[0,0])
        yxt += segcov
        #del segcov

        x_m += tf.linalg.matvec(mat,x,transpose_a=True)
        y_m += tf.linalg.matvec(segmat,y,transpose_a=True)

        n += 1
    xxt = xxt - tf.matmul([x_m],[x_m],transpose_a=True)/n
    yxt = yxt - tf.matmul([x_m],[y_m],transpose_a=True)/n
    inverse_xxt = tf.linalg.pinv(xxt)
    A = tf.matmul(inverse_xxt,yxt)
    b = (y_m - tf.linalg.matvec(A,x_m,transpose_a=True))/n
    A_filter = tf.reshape(A,(1,1,seg_channels,seg_channels))
    return A_filter,b

In [5]:
def connected_calculate_A_and_b(imghead, seghead, img_train,seg_train):
    imgflat = np.prod(imghead(next(iter(img_train))[0]).shape)
    segflat = np.prod(seghead(next(iter(seg_train))[0]).shape)
    end_shape = next(iter(seg_train))[0].shape
    n = 0.0

    xxt = np.zeros([imgflat])
    yxt = np.zeros([segflat])
    x = np.zeros([imgflat])
    y = np.zeros([segflat]) 

    bar = tqdm.notebook.tqdm(total = int(img_train.cardinality()))

    for item in iter(zip(img_train,seg_train)):

        bar.update(1)

        image = item[0][0]
        segmentation = item[1][0]

        imgdecom = imghead(image)
        segdecom = seghead(segmentation)

        mat = tf.reshape(imgdecom,[-1])
        segmat = tf.reshape(segdecom,[-1])

        cov = tf.matmul([mat],[mat],transpose_a=True)
        xxt += cov
        segcov = tf.matmul([mat],[segmat],transpose_a=True)
        yxt += segcov
        x+=mat
        y+=segmat
        n += 1
        
    xxt = xxt - tf.matmul([x],[x],transpose_a=True)/n
    yxt = yxt - tf.matmul([x],[y],transpose_a=True)/n
    inverse_xxt = tf.linalg.pinv(xxt)
    A = tf.linalg.matmul(inverse_xxt,yxt)
    b = (y - tf.linalg.matvec(A,x,transpose_a=True))/n
    return A,b

In [6]:
def build_model_instance(train,
                test,
                dataset,
                model_name,
                keep_percent=1.0,
                count=3,
                sample_size=100,
                activity_regularizer=None,
                inverse_activity_regularizer=None,
                activation_before=False,
                check_build=False):
    
    stats = None
    
    broker = mb.ModelBroker(trainset=train,
                                testset=test,
                                dirname=dataset+"_"+model_name,
                                keep_percent=keep_percent,
                                count=count,
                                sample_size=sample_size,
                                activity_regularizer = activity_regularizer,
                                inverse_activity_regularizer=inverse_activity_regularizer,
                                activation_before=activation_before)
    
    head,invhead = broker.build_model()
    head,invhead = broker.load_model()    
    if check_build:
        train_psnr,train_ncc = broker.check_build(head,invhead,train,stats_only=True)
        test_psnr,test_ncc = broker.check_build(head,invhead,test,stats_only=True)
        stats = (train_psnr,train_ncc,test_psnr,test_ncc)
    return head,invhead, stats

In [7]:
def conv_metric_calculate(img_ds,seg_ds,A_filter,b):
    threshold_intensity = 0.03
    dice_coeff_vals = []
    iou_coeff_vals = []
    n = 0
    
    for image,seg_base in iter(zip(img_ds,seg_ds)):
        imgdecom = imghead(image[0])
        conv = tf.nn.conv2d(imgdecom, A_filter,1,"VALID")
        conv = tf.nn.bias_add(conv,b)
        seg = seginvhead(conv)
        y_true = tf.cast(tf.reduce_min(seg_base[0],2)==0,tf.float64)
        y_pred = tf.cast(tf.reduce_min(seg[0],2)<threshold_intensity,tf.float64)
        dice_coeff_vals.append(dice_coef(y_true,y_pred))
        iou_coeff_vals.append(iou_coef(y_true,y_pred))
        n+=1
    return iou_coeff_vals,dice_coeff_vals,n


In [8]:
def connected_metric_calculate(img_ds,seg_ds,A,b):
    threshold_intensity = 0.01
    dice_coeff_vals = []
    iou_coeff_vals = []
    n = 0
    reconstruct = seghead(next(iter(seg_ds))[0]).shape
    for image,seg_base in iter(zip(img_ds,seg_ds)):
        imgdecom = imghead(image[0])
        imgdecom = tf.reshape(imgdecom,(1,-1))
        segdecom = tf.linalg.matvec(A,imgdecom,transpose_a=True)+b
        seg = seginvhead(tf.reshape(segdecom,(reconstruct)))
        y_true = tf.cast(tf.reduce_min(seg_base[0],2)==0,tf.float64)
        y_pred = tf.cast(tf.reduce_min(seg[0],2)<threshold_intensity,tf.float64)
        dice_coeff_vals.append(dice_coef(y_true,y_pred))
        iou_coeff_vals.append(iou_coef(y_true,y_pred))
        n+=1
    return iou_coeff_vals,dice_coeff_vals,n

In [9]:
def calculate_metrics(seg_ds,img_ds,imghead,seghead,seginvhead,A,b,mode):
    if mode == "conv":
        iou_coeff_vals,dice_coeff_vals,n = conv_metric_calculate(img_ds,seg_ds,A,b)
    else:
        iou_coeff_vals,dice_coeff_vals,n = connected_metric_calculate(img_ds,seg_ds,A,b)
    dice_coeff_mean = sum(dice_coeff_vals)/n
    iou_coeff_mean = sum(iou_coeff_vals)/n
    dice_coeff_std = (sum([((x - dice_coeff_mean) ** 2) for x in dice_coeff_vals]) / n)**0.5
    iou_coeff_std = (sum([((x - iou_coeff_mean) ** 2) for x in iou_coeff_vals]) / n)**0.5
    return dice_coeff_mean, iou_coeff_mean, dice_coeff_std, iou_coeff_std

In [10]:
def dice_coef(y_true, y_pred,smooth=1):
    y_true_f = tf.reshape(y_true,-1)
    y_pred_f =tf.reshape(y_pred,-1)
    intersection = tf.reduce_sum(y_true_f * y_pred_f,0)

    return float((2. * intersection+smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f)+smooth))

In [11]:
def iou_coef(y_true, y_pred,smooth=1):
  intersection = tf.reduce_sum(y_true * y_pred, 0)
  union = tf.reduce_sum(y_true,0)+tf.reduce_sum(y_pred,0)-intersection
  iou = tf.reduce_mean((intersection+1) / (union+1), 0)
  return float(iou)

In [30]:
counts_exp = [("count",i) for i in range(4,6)]
keep_percents_exp = [("keep_percent",i/10) for i in range(4,12,2)]
train_sizes_exp = [("train_size",i*2000) for i in range(1,3)] + [("train_size",None)]
res_exp = [("res",2**i) for i in range(6,9,2)]
counts_exp = [("count",i) for i in range(4,6)]
experiments = [("skip",0)] + keep_percents_exp# + train_sizes_exp + counts_exp +  res_exp 

In [31]:
experiments

[('skip', 0),
 ('keep_percent', 0.4),
 ('keep_percent', 0.6),
 ('keep_percent', 0.8),
 ('keep_percent', 1.0)]

In [ ]:
#change mode between conv and connected to explore differenet linear inverse methods
for mode in ["conv","connected"]:
    test_size = 300

    dataset = "pets"

    nudge = np.random.randint(0,1000000)

    if mode == "conv":
        calculate_A_and_b = conv_calculate_A_and_b
    else:
        calculate_A_and_b = connected_calculate_A_and_b

    df = pd.DataFrame()

    save_name = mode+"_formal_experiment_"+str(nudge)
    print(f"saving results to:{save_name}")
    for settings in tqdm.notebook.tqdm(experiments):

        #These are the standard experiment settings
        activity_regularizer = scaledtanh
        inverse_activity_regularizer = scaledatanh
        count = 3
        keep_percent = 0.1
        train_size = None
        res = 128

        #Initialise the experiment record for this iteration
        record = pd.Series()

        #Change the values of the paramaters passed into the model builder and data loader
        variable, value = settings
        if variable == "count":
            count = value
        if variable == "keep_percent":
            keep_percent = value
        if variable == "train_size":
            train_size = value
        if variable == "res":
            res = value

        #Load the datasets
        loader = dl.DataLoader(IMAGE_SIZE=res,dataset=dataset,take=train_size)
        img_ds = loader.import_processed_img()
        seg_ds = loader.import_processed_seg()
        cardinality = int(img_ds.cardinality())

        #Generate train/test splits
        img_test = img_ds.take(test_size)
        seg_test = seg_ds.take(test_size)
        img_train = img_ds.skip(test_size)
        seg_train = seg_ds.skip(test_size)    

        #Record experiment parameters
        record["resolution"]=res
        record["count"]=count
        record["keep_percent"] = keep_percent
        record["activity_regularizer"] = activity_regularizer != None
        record["training_data_size"] = train_size

        #Training image reconstruction network
        img_train_start = time.time()
        imghead, imginvhead,stats = build_model_instance(img_train,img_test,dataset,"img",keep_percent = keep_percent,count=count,check_build=True)
        psnr_train,ncc_train,psnr_test,ncc_test = stats
        img_train_end = time.time()

        #Record values from image reconstruction network experiements
        record["img_channel_size"] = imghead(next(iter(img_train))[0]).shape[-1]
        record["img_train_time"] = img_train_end - img_train_start
        record["train_img_psnr"] = psnr_train
        record["train_img_ncc"] = ncc_train
        record["test_img_psnr"] = psnr_test
        record["test_img_ncc"] = ncc_test

        #Training segmentation reconstruction network
        seg_train_start = time.time()
        seghead, seginvhead,stats = build_model_instance(seg_train,seg_test,dataset,"seg",count=count,keep_percent = keep_percent,check_build=True)
        psnr_train,ncc_train,psnr_test,ncc_test = stats
        seg_train_end = time.time()

        #Record values from segmentation reconstruction network experiements
        record["seg_channel_size"] = seghead(next(iter(seg_train))[0]).shape[-1]
        record["seg_train_time"] = img_train_end - img_train_start
        record["train_seg_psnr"] = psnr_train
        record["train_seg_ncc"] = ncc_train
        record["test_seg_psnr"] = psnr_test
        record["test_seg_ncc"] = ncc_test

        #Calculate A and b
        train_time_start = time.time()
        A,b = calculate_A_and_b(imghead,seghead,img_train,seg_train)
        train_time_end = time.time()
        record["linear_inverse_train_time"] = train_time_end - train_time_start 

        #Caculate metrics of image segmentation network
        dice_mean_test, iou_mean_test, dice_std_test, iou_std_test = calculate_metrics(seg_test,img_test,imghead,seghead,seginvhead,A,b,mode)
        dice_mean_train, iou_mean_train, dice_std_train, iou_std_train = calculate_metrics(seg_train,img_train,imghead,seghead,seginvhead,A,b,mode)

        #Record metrics
        record["dice_mean_train"] = dice_mean_train
        #record["iou_mean_train"] = iou_mean_train
        record["dice_std_train"] = dice_std_train
        #record["iou_std_train"] = iou_std_train
        record["dice_mean_test"] = dice_mean_test
        #record["iou_mean_test"] = iou_mean_test
        record["dice_std_test"] = dice_std_test
        #record["iou_std_test"] = iou_std_test

        #Append experiment to dataframe, and save most recent dataframe
        df = df.append(record,ignore_index=True)
        df.to_csv(save_name)

saving results to:conv_formal_experiment_54912


  0%|          | 0/5 [00:00<?, ?it/s]

keep_percent 0.20629283704945683


<ipython-input-33-5eb84df89684>:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  record = pd.Series()


meanimg.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
Starting level 0
Completing 64.0
pca shape tf.Tensor([27 27], shape=(2,), dtype=int32)
keep_channels 5 keep_max 12.0
keep_channels 5
ufilts.shape (1, 1, 1, 27, 5)
end loop 64.0
Starting level 1
Completing 32.0
pca shape tf.Tensor([45 45], shape=(2,), dtype=int32)
keep_channels 9 keep_max 80.0
keep_channels 9
ufilts.shape (1, 1, 1, 45, 9)
end loop 32.0
Starting level 2
Completing 16.0
pca shape tf.Tensor([81 81], shape=(2,), dtype=int32)
keep_channels 16 keep_max 576.0
keep_channels 16
ufilts.shape (1, 1, 1, 81, 16)
end loop 16.0
saving to: models/pets_img
out.shape (1, 16, 16, 16)
keep_percent 0.20629283704945683
meanimg.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
Starting level 0
Completing 64.0
pca shape tf.Tensor([27 27], shape=(2,), dtype=int32)
keep_channels 5 keep_max 12.0
keep_channels 5
ufilts.shape (1, 1, 1, 27, 5)
end loop 6

...

keep_percent 0.374858962356333
meanimg.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
Starting level 0
Completing 64.0
pca shape tf.Tensor([27 27], shape=(2,), dtype=int32)
keep_channels 10 keep_max 12.0
keep_channels 10
ufilts.shape (1, 1, 1, 27, 10)
end loop 64.0
Starting level 1
Completing 32.0
pca shape tf.Tensor([90 90], shape=(2,), dtype=int32)
keep_channels 33 keep_max 160.0
keep_channels 33
ufilts.shape (1, 1, 1, 90, 33)
end loop 32.0
Starting level 2
Completing 16.0
pca shape tf.Tensor([297 297], shape=(2,), dtype=int32)
keep_channels 111 keep_max 2112.0
keep_channels 111
ufilts.shape (1, 1, 1, 297, 111)
end loop 16.0
saving to: models/pets_img
out.shape (1, 16, 16, 111)
keep_percent 0.374858962356333
meanimg.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
Starting level 0
Completing 64.0
pca shape tf.Tensor([27 27], shape=(2,), dtype=int32)
keep_channels 10 keep_max 12.0
keep_channe

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])>]
Consider rewriting this model with the Functional API.


array([[[0.31272358, 0.32361558, 0.29820484],
        [0.31193513, 0.32502395, 0.29021668],
        [0.3099868 , 0.33001161, 0.29664665],
        ...,
        [0.28866425, 0.25707817, 0.21427792],
        [0.28846318, 0.24594535, 0.19817851],
        [0.2431497 , 0.240153  , 0.19564041]],

       [[0.32199422, 0.33907256, 0.29442403],
        [0.31503335, 0.32776743, 0.288838  ],
        [0.32305023, 0.34004864, 0.300248  ],
        ...,
        [0.26030803, 0.19661602, 0.14610907],
        [0.24157429, 0.20276742, 0.15124559],
        [0.23732623, 0.20338351, 0.18857566]],

       [[0.33531091, 0.34315404, 0.29845428],
        [0.33677572, 0.34288642, 0.30244812],
        [0.35008377, 0.35924432, 0.33313182],
        ...,
        [0.24312434, 0.1840155 , 0.13597628],
        [0.26333824, 0.1853219 , 0.14950694],
        [0.23001255, 0.20721652, 0.1820188 ]],

       ...,

       [[0.48685852, 0.49200368, 0.28987777],
        [0.49323493, 0.53634971, 0.29418996],
        [0.57753813, 0

array([[[0.31272358, 0.32361558, 0.29820484],
        [0.31193513, 0.32502395, 0.29021668],
        [0.3099868 , 0.33001161, 0.29664665],
        ...,
        [0.28866425, 0.25707817, 0.21427792],
        [0.28846318, 0.24594535, 0.19817851],
        [0.2431497 , 0.240153  , 0.19564041]],

       [[0.32199422, 0.33907256, 0.29442403],
        [0.31503335, 0.32776743, 0.288838  ],
        [0.32305023, 0.34004864, 0.300248  ],
        ...,
        [0.26030803, 0.19661602, 0.14610907],
        [0.24157429, 0.20276742, 0.15124559],
        [0.23732623, 0.20338351, 0.18857566]],

       [[0.33531091, 0.34315404, 0.29845428],
        [0.33677572, 0.34288642, 0.30244812],
        [0.35008377, 0.35924432, 0.33313182],
        ...,
        [0.24312434, 0.1840155 , 0.13597628],
        [0.26333824, 0.1853219 , 0.14950694],
        [0.23001255, 0.20721652, 0.1820188 ]],

       ...,

       [[0.48685852, 0.49200368, 0.28987777],
        [0.49323493, 0.53634971, 0.29418996],
        [0.57753813, 0

keep_percent 0.374858962356333
meanimg.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
Starting level 0
Completing 64.0
pca shape tf.Tensor([27 27], shape=(2,), dtype=int32)
keep_channels 10 keep_max 12.0
keep_channels 10
ufilts.shape (1, 1, 1, 27, 10)
end loop 64.0
Starting level 1
Completing 32.0
pca shape tf.Tensor([90 90], shape=(2,), dtype=int32)
keep_channels 33 keep_max 160.0
keep_channels 33
ufilts.shape (1, 1, 1, 90, 33)
end loop 32.0
Starting level 2


In [26]:
peformance_df = df[["training_data_size","count","keep_percent","resolution",'dice_mean_test', 'dice_mean_train',
       'dice_std_test', 'dice_std_train',"linear_inverse_train_time","img_train_time","seg_train_time","img_channel_size"]]

In [27]:
peformance_df.replace({np.nan:int(img_train.cardinality())})

,training_data_size,count,keep_percent,resolution,dice_mean_test,dice_mean_train,dice_std_test,dice_std_train,linear_inverse_train_time,img_train_time,seg_train_time,img_channel_size
0,7049,4.0,0.1,128.0,0.603597,0.596743,0.181516,0.182701,69.234996,117.951081,117.951081,65.0
1,7049,4.0,0.2,128.0,0.603490,0.596670,0.181426,0.182620,85.138880,142.841959,142.841959,149.0
2,7049,4.0,0.4,128.0,0.603496,0.596668,0.181442,0.182621,106.014822,186.633168,186.633168,283.0
3,7049,4.0,0.6,128.0,0.603429,0.596622,0.181405,0.182585,143.232454,276.932203,276.932203,433.0
4,7049,4.0,0.8,128.0,0.335789,0.322221,0.244324,0.252329,247.535285,307.982901,307.982901,586.0


In [ ]:
peformance_df[["dice_mean_test","dice_mean_train"]].boxplot()

In [ ]:
peformance_df[["dice_std_test","dice_std_train"]].boxplot()

In [ ]:

reconstruct = seghead(next(iter(seg_train))[0]).shape
skip = np.random.randint(0,70)
image,seg_base = next(iter(zip(img_test.skip(skip),seg_test.skip(skip))))
imgdecom = imghead(image[0])
imgdecom = tf.reshape(imgdecom,(1,-1))
segdecom = tf.linalg.matvec(A,imgdecom,transpose_a=True)+b
seg = seginvhead(tf.reshape(segdecom,(reconstruct)))
y_true = tf.cast(tf.reduce_min(seg_base[0],2)==0,tf.float64)
y_pred = tf.cast(tf.reduce_min(seg[0],2)<threshold_intensity,tf.float64)
plt.subplot(2,1,1)
plt.imshow(np.hstack([image[0],seg_base[0],seg[0]]))
plt.subplot(2,1,2)
plt.imshow(np.hstack([y_true,y_pred]))


In [ ]:

skip = np.random.randint(0,70)
image,seg_base = next(iter(zip(img_test.skip(skip),seg_test.skip(skip))))
imgdecom = imghead(image[0])
conv = tf.nn.conv2d(imgdecom, A,1,"VALID")
conv = tf.nn.bias_add(conv,b)
seg = seginvhead(conv)
y_true = tf.cast(tf.reduce_min(seg_base[0],2)==0,tf.float64)
y_pred = tf.cast(tf.reduce_min(seg[0],2)<threshold_intensity,tf.float64)
plt.subplot(2,1,1)
plt.imshow(np.hstack([image[0],seg_base[0],seg[0]]))
plt.subplot(2,1,2)
plt.imshow(np.hstack([y_true,y_pred]))